In [6]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import traceback
import logging
import pyspark
from pyspark.sql import SparkSession
import TableColumn
from datetime import datetime
import Schemas
from SQLConnection import SQLConnection 
from datetime import datetime
from TransactionLog import TransactionLog
import random
import string

ModuleNotFoundError: No module named 'DataPipeline'

In [2]:
df = pd.read_excel("D:\\OneDrive - Cipla Limited\\Mayur Data\\MLOPS_End_to_End\\DataSourcesFiles\\APQR_ALL_SITES\\APQR Indore 4.XLSX")


In [167]:
df1 = df.copy()

In [168]:
df1.shape

(151481, 20)

In [172]:
df1.columns = [i.upper().replace(' ', '_') for i in df1.columns]

In [178]:
# df1['LONG_TEXT_DESCRIPTION'] = df1['LONG_TEXT_DESCRIPTION'].apply(lambda x : x.upper())
# df1['MIC_DESCRIPTION'] = df1['MIC_DESCRIPTION'].apply(lambda x : x.upper())

In [179]:
df1.columns

Index(['INSPECTION_LOT_NO.', 'MATERIAL_CODE', 'MATERIAL_DESCRIPTION', 'PLANT',
       'PLANT_NAME', 'BATCH_NO.', 'RECIPE_NO.', 'PRODUCTION_VERSION',
       'UD_STATUS', 'OPERATION_NO.', 'CHAR_SR._NO.', 'MIC_NO',
       'MIC_DESCRIPTION', 'TARGET_VALUE', 'LOWER_VALUE', 'HIGHER_VALUE',
       'RESULT', 'UOM', 'LONG_TEXT_DESCRIPTION', 'ADDITIONAL_INFORMATION'],
      dtype='object')

In [180]:
batchno = [31004737,31007552,31002162,31000009,31000471,31000137,31000071,31000087,31003578,31006159,31006043,31000626,31000731,31000732,
31004467,31004468,31006891,31001500,31001501,
31004737,31001637,31007031,31002214,31002718,31002710,31002717,31002727,31002846,31003057,31003059,31004274,
31003732,31000631,31003103,31004735,31003917,31002180,31005128,
31004736,31002178,32008420,31001398,31006003
]

In [181]:
col = ["% FINES","LOSS ON DRYING","BULK DENSITY","TAPPED DENSITY","GRANULATION YIELD","COMPRESSION YIELD","COATING YIELD","MANUFACTURING YIELD"]
df1 = df1[df1["MIC_DESCRIPTION"].isin(col)]
df1 = df1[df1["MATERIAL_CODE"].isin(batchno)]

In [183]:
df1['MATERIAL_CODE'].nunique()

40

In [184]:
df1.shape

(40962, 20)

In [186]:
df1 = df1[((df1["MIC_DESCRIPTION"] == 'COMPRESSION YIELD') & (df1['UOM'] == '%')) | 
    ((df1["MIC_DESCRIPTION"] == 'LOSS ON DRYING') & (df1['UOM'].isin(['%','%W/W']))) |
    ((df1["MIC_DESCRIPTION"] == 'BULK DENSITY') & (df1['UOM'].isin(['%','gm/ml']))) |
    ((df1["MIC_DESCRIPTION"] == 'TAPPED DENSITY') & (df1['UOM'].isin(['%','gm/ml']))) |
    ((df1["MIC_DESCRIPTION"] == '% FINES') & (df1['UOM'].isin(['%','%W/W']))) |
    ((df1["MIC_DESCRIPTION"] == 'COATING YIELD') & (df1['UOM'] == '%')) |
    ((df1["MIC_DESCRIPTION"] == 'GRANULATION YIELD') & (df1['UOM'] == '%'))
    ]

In [187]:
df1.shape

(34916, 20)

In [188]:
df2 = pd.DataFrame(df1.groupby(['MATERIAL_CODE','BATCH_NO.', 'MIC_DESCRIPTION']).size() == 1).reset_index()

In [189]:
df2[0].value_counts()

True     26131
False     1117
Name: 0, dtype: int64

In [190]:
false_product = list(df2[df2[0] == False]['MATERIAL_CODE'])

In [191]:
test = pd.DataFrame(df1.groupby(['MATERIAL_CODE','BATCH_NO.', 'MIC_DESCRIPTION']).size()).reset_index()
not_included_batch = set(test[test[0]>1]['MATERIAL_CODE'])

In [192]:
df1 = df1[~df1['MATERIAL_CODE'].isin(not_included_batch)]

In [193]:
pd.DataFrame(df1.groupby(['MATERIAL_CODE','BATCH_NO.', 'MIC_DESCRIPTION']).size() == 1).reset_index()[0].value_counts()

True    23369
Name: 0, dtype: int64

In [194]:
df1[df1['MATERIAL_CODE'].isin(false_product)]

Empty DataFrame
Columns: [INSPECTION_LOT_NO., MATERIAL_CODE, MATERIAL_DESCRIPTION, PLANT, PLANT_NAME, BATCH_NO., RECIPE_NO., PRODUCTION_VERSION, UD_STATUS, OPERATION_NO., CHAR_SR._NO., MIC_NO, MIC_DESCRIPTION, TARGET_VALUE, LOWER_VALUE, HIGHER_VALUE, RESULT, UOM, LONG_TEXT_DESCRIPTION, ADDITIONAL_INFORMATION]
Index: []

In [195]:
df1['MATERIAL_CODE'].nunique()

30

In [197]:
df_pivot = pd.pivot(df1,index = ['BATCH_NO.','MATERIAL_CODE','MATERIAL_DESCRIPTION','PLANT','PLANT_NAME'], columns = ["MIC_DESCRIPTION"], values = ['RESULT' ])

In [198]:
df_pivot = df_pivot['RESULT'].reset_index()

In [201]:
from datetime import datetime
import numpy as np
df_pivot = df_pivot.rename(columns = {'MATERIAL_DESCRIPTION' : 'PRODUCTNAME' , '% FINES' : 'FINES_STATION_1' , 'BULK DENSITY' : 'BULK_DENSITY_STATION_1' , 'LOSS ON DRYING' : 'LOSS_ON_DRYING_STATION_1' , 'TAPPED DENSITY' :  'TAPPED_DENSITY_STATION_1' ,  'COATING YIELD' : 'COATING_YIELD_PERCENT' , 'COMPRESSION YIELD' : 'COMPRESSION_YIELD_PERCENT' , 'GRANULATION YIELD' :  'GRANULATION_YIELD_PERCENT'})
site_dict = {1028 : {'SITE' : 'Indore' , 'UNIT' : '4'}}

df_pivot['SITE'] = df_pivot['PLANT'].apply(lambda x : site_dict[x]['SITE'])
df_pivot['UNIT'] = df_pivot['PLANT'].apply(lambda x : site_dict[x]['UNIT'])


df_pivot['CURRENT_DATETIME'] = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
df_pivot["CURRENT_DATETIME"] = pd.to_datetime(df_pivot["CURRENT_DATETIME"])

df_pivot['BATCH_SIZE'] = np.nan

df_pivot = df_pivot.drop(['PLANT' , 'PLANT_NAME'] , axis=1)

df_pivot['LINE'] = np.nan
df_pivot['FINES_STATION_2'] = np.nan 
df_pivot['BULK_DENSITY_STATION_2']= np.nan
df_pivot['LOSS_ON_DRYING_STATION_2']= np.nan
df_pivot['TAPPED_DENSITY_STATION_2']= np.nan
df_pivot['MANUFACTURING_YIELD_PERCENT']= np.nan
df_pivot['MACHINECODE'] = np.nan


#df_pivot.columns
col = ['BATCH_ID', 'PRODUCTCODE', 'PRODUCTNAME', 'CURRENT_DATETIME',
    'SITE', 'UNIT','LINE','BATCH_SIZE','FINES_STATION_1',
    'BULK_DENSITY_STATION_1', 'LOSS_ON_DRYING_STATION_1', 'TAPPED_DENSITY_STATION_1',  'FINES_STATION_2',
    'BULK_DENSITY_STATION_2', 'LOSS_ON_DRYING_STATION_2',
    'TAPPED_DENSITY_STATION_2', 'COMPRESSION_YIELD_PERCENT', 'GRANULATION_YIELD_PERCENT',
    'COATING_YIELD_PERCENT','MANUFACTURING_YIELD_PERCENT' ]

df_pivot = df_pivot[col]

for i in ['FINES_STATION_1','BULK_DENSITY_STATION_1', 'LOSS_ON_DRYING_STATION_1', 'TAPPED_DENSITY_STATION_1',  'FINES_STATION_2',
    'BULK_DENSITY_STATION_2', 'LOSS_ON_DRYING_STATION_2','TAPPED_DENSITY_STATION_2', 'COMPRESSION_YIELD_PERCENT', 'GRANULATION_YIELD_PERCENT',
    'COATING_YIELD_PERCENT','MANUFACTURING_YIELD_PERCENT' ]:
    
    df_pivot[i] = df_pivot[i].astype(np.float64)

KeyError: "['BATCH_ID', 'PRODUCTCODE'] not in index"

In [83]:
df_pivot_N.to_excel("D:\\OneDrive - Cipla Limited\\Mayur Data\\MLOPS_End_to_End\\DataSourcesFiles\\APQR_ALL_SITES\\Processed_Indore4.xlsx")